In [2]:
import intprim
from intprim.probabilistic_movement_primitives import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import random
import pandas as pd
from frechetdist import frdist
import math
from cem import CEM
from utils import *

from two_dof.toy_robot import ToyRobot
from two_dof.toy_obstacle import ToyObstacle

In [12]:
%matplotlib
dataset= np.load('/home/zhiyuan/notebook_script/0930/l_shape.npy')
# dataset=dataset[:,70:,]
fig, ax = plt.subplots()
ax.axis([0, 10, 0, 10])
plt.gca().set_aspect(1)
for i in range(1):
    ax.plot(dataset[i,:,0], dataset[i,:,1],'g-',alpha=0.3)
#add rectangle to plor
ax.add_patch(Rectangle((5, 3), 1, 1,

             angle=0,
             edgecolor = 'none',
             facecolor = 'blue',
             fill=True,
             lw=1))


plt.show() 

Using matplotlib backend: QtAgg


In [11]:
toy_obstacle = ToyObstacle(5.5, 3.5, 1, 1,safe_distance=0.2)

ax = toy_obstacle.visualize(ax=None)
ax.axis([0, 10, 0, 10])
plt.gca().set_aspect(1)
for i in range(30):
    ax.plot(dataset[i,:,0], dataset[i,:,1],'g-',alpha=0.3)
plt.show()

In [ ]:
toy_robo = ToyRobot(6., 8., 0., 0.)

In [50]:
# data2dof= np.zeros([30, 100, 2])
# for i in range(30):
    
#     data2dof[i,:,0] = toy_robo.ik(dataset[i,:,0], dataset[i,:,1])[0]
#     data2dof[i,:,1] = toy_robo.ik(dataset[i,:,0], dataset[i,:,1])[1]

In [52]:
dataset= np.load('/home/zhiyuan/notebook_script/cem_promp/data2dof.npy')
num_joints =2
# Create a ProMP with Gaussian basis functions.

basis_model = intprim.basis.GaussianModel(8, 0.1, ["x","y"])
promp = ProMP(basis_model)
Q = dataset.transpose(0,2,1)

# Add Demonstrations to the ProMP, which in turn calculates the list of weights for each demonstration.
for i in range(len(Q)):
	promp.add_demonstration(Q[i])

n_samples = 30# Number of trajectoies to sample
domain = np.linspace(0,1,100)

# sample some trajs
samples_learn= []
for i in range(n_samples):
    samples, _ = promp.generate_probable_trajectory(domain)
    samples_learn.append(samples)
samples_learn=np.array(samples_learn)  


mean_margs = np.zeros(samples.shape)


upper_bound =  np.zeros([2,100])
lower_bound =  np.zeros([2,100])
sigma_s = np.zeros([2,2,100])
for i in range(len(domain)):
    mu_marg_q, Sigma_marg_q = promp.get_marginal(domain[i])
    sigma_s[:,:,i] = Sigma_marg_q
    std_q = np.diagonal(Sigma_marg_q) ** 0.5

    mean_margs[:,i] = mu_marg_q
    upper_bound[:,i] = mu_marg_q + std_q
    lower_bound[:,i] = mu_marg_q - std_q


In [70]:
mean_margs[:,30]

array([-0.37431626,  2.00549582])

In [71]:
mean_margs[:,70] 

array([-0.60741837,  2.32964738])

In [58]:
th1=mean_margs[0,:]
th2=mean_margs[1,:]
x, y = toy_robo.fk(th1, th2)

In [67]:
x, y = toy_robo.fk(mean_margs[0,:], mean_margs[1,:])
cond_traj = np.array([x,y])

In [80]:
mean_margs.shape

(2, 100)

In [73]:
mean_margs.max()


2.7261004450534756

In [74]:
mean_margs.min()

-1.1429127753883108

In [ ]:
0.03087525  1.57015466  0.25613693 -0.14906376  2.55112826  0.71432574

In [76]:
th1=np.array([-0.37909077,0.75035802 ])
th2=np.array([1.60758344,2.51441848])
x, y = toy_robo.fk(th1, th2)

print(x)
print(y)

[ 8.25927293 -3.55071098]
[5.31541572 3.10842369]


In [85]:
via_point= np.array([-0.81039905 , 2.49865885  ,0.11564862, -0.66045037,  2.41744664 , 0.65774791])
def joint2task(via_point):
    q = np.zeros(6)
    x0, y0 = toy_robo.fk(via_point[0], via_point[1])
    x1, y1 = toy_robo.fk(via_point[3], via_point[4])
    q[0]= x0
    q[1]=y0
    q[2]= via_point[2]
    q[3]=x1
    q[4]=y1
    q[5]=via_point[5]
    return q

q =joint2task(via_point)
print(q)


[3.19770779 3.59749919 0.11564862 3.25728918 4.18088307 0.65774791]


In [65]:
fig, ax = plt.subplots()
ax.axis([0, 10, 0, 10])
plt.gca().set_aspect(1)
ax.plot(domain, y, 'g', alpha=0.3)
ax.plot(domain, x, 'r', alpha=0.3)
#add rectangle to plor
ax.add_patch(Rectangle((5, 3), 1, 1,
# ax.add_patch(Rectangle((4, 5), 3, 3,
             angle=0,
             edgecolor = 'none',
             facecolor = 'blue',
             fill=True,
             lw=1))


plt.show()
print(dataset.shape)

(30, 100, 2)


In [ ]:
# toy_obstacle = ToyObstacle(5.5, 3.5, 1, 1,safe_distance=0.2)
def get_reward(via_point):
#     task space转为joint space
#     v_joint = toy_robo.ik(dataset[i,:,0], dataset[i,:,1])[0]
    
    old_promp=promp
    via_point = via_point.reshape([-1,3])
    t_cond = np.zeros(2+via_point.shape[0])
    t_cond[0]=0
    t_cond[-1] = 1
    for i in range(via_point.shape[0]):
        t_cond[i+1]=via_point[i,2]

    q_cond =np.zeros([2+via_point.shape[0],2])
    q_cond[0]= mean_margs[:,0]
    q_cond[-1]= mean_margs[:,-1]
    for i in range(via_point.shape[0]):
        q_cond[i+1]=via_point[i,:2]

    mu_w_cond_rec, Sigma_w_cond_rec=old_promp.get_basis_weight_parameters()

    for i in range(t_cond.shape[0]):
        mu_w_cond_rec, Sigma_w_cond_rec = old_promp.get_conditioned_weights(t_cond[i], q_cond[i], mean_w=mu_w_cond_rec, var_w=Sigma_w_cond_rec)

    cond_mean = np.zeros([2,100])
    for i in range(len(domain)):
        mu_marg_q_con, Sigma_marg_q_con = promp.get_marginal(domain[i], mu_w_cond_rec, Sigma_w_cond_rec)
        cond_mean[:,i] = mu_marg_q_con
    
    
    x, y = toy_robo.fk(cond_mean[0,:], cond_mean[1,:])
    cond_traj = np.array([x,y])
    distance = toy_obstacle.eval_distance(x,y) 
        
        
    
    col=  collision_detect(limit,cond_traj.T)
    obs_dis= traj_rect_dist(cond_traj.T,  limit)
    if obs_dis > 0.3:
        dis_reward = 3
    else:
        dis_reward = -0.5*obs_dis #0.2 is good for dtw


    # if obs_dis > 0.3:
    #     dis_reward = 3
    # # elif obs_dis < 0.1:
    # #     dis_reward = 3
    # else:
    #     dis_reward =0.5*obs_dis #0.2 is good for dtw

    # cond_traj3 = cond_traj[:,1:99:2]    
    # cond_traj3 = cond_traj[:,1:99:3]
    fre_dis= get_f_dist(ref_traj_T,cond_traj)

    # fre_dis= get_f_dist(ref3_t,cond_traj3)

    # dtw_cost= get_dtw_cost(ref3_t,cond_traj3)
    reward =  fre_dis  +10 *col + dis_reward
    # reward =  dtw_cost  +10 *col + dis_reward
    # reward =  dtw_cost  +10 *col
    # reward =  fre_dis   + dis_reward
    # reward =  fre_dis   +10*col 
    return reward
    # return reward, fre_dis,col,dis_reward